<a href="https://colab.research.google.com/github/MJMortensonWarwick/BAV/blob/main/Discrete_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discrete Optimisation in PuLP

Optimising school dinners (adapted from: https://github.com/tirthajyoti)

In [1]:
# install packages - note some are Linux installs
!pip install pulp                  # PuLP
!sudo apt-get install glpk-utils  # GLPK
!sudo apt-get install coinor-cbc  # CoinOR

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 6.8 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 692 kB of archives.
After this operation, 1,664 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19

Our problem is to optimise the cost of school dinner provisions while keeping to a minimum level of nutrition. As such it belongs in the category of discrete optimisation (as we have a discrete set of options to pick from).

We'll start by importing the package (PuLP) and the data:

In [2]:
import pandas as pd
from pulp import *

# upload a file into Colab - "school_food.xlsx" from the Moodle.
from google.colab import files
files.upload()

Saving school_food.csv to school_food.csv


{'school_food.csv': b'\xef\xbb\xbfFoods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)\r\nFrozen Broccoli,0.48,10 Oz Pkg,73.8,0,0.8,68.2,13.6,8.5,8,5867.4,160.2,159,2.3\r\nFrozen Corn,0.54,1/2 Cup,72.2,0,0.6,2.5,17.1,2,2.5,106.6,5.2,3.3,0.3\r\nRaw Lettuce Iceberg,0.06,1 Leaf,2.6,0,0,1.8,0.4,0.3,0.2,66,0.8,3.8,0.1\r\n Baked Potatoes,0.18,1/2 Cup,171.5,0,0.2,15.2,39.9,3.2,3.7,0,15.6,22.7,4.3\r\nTofu,0.93,1/4 block,88.2,0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2\r\nRoasted Chicken,2.52,1 lb chicken,277.4,129.9,10.8,125.6,0,0,42.2,77.4,0,21.9,1.8\r\nSpaghetti W/ Sauce,2.34,1 1/2 Cup,358.2,0,12.3,1237.1,58.3,11.6,8.2,3055.2,27.9,80.2,2.3\r\nRaw Apple,0.72,"1 Fruit,3/Lb,Wo/Rf",81.4,0,0.5,0,21,3.7,0.3,73.1,7.9,9.7,0.2\r\nBanana,0.45,"1 Fruit,Wo/Skn&Seeds",104.9,0,0.5,1.1,26.7,2.7,1.2,92.3,10.4,6.8,0.4\r\nWheat Bread,0.15,1 Sl,65,0,1,134.5,12.4,1.3,2.2,0,0,10.8,0.7\r\nWhit

In [3]:
df = pd.read_csv("/content/school_food.csv") # Colab saves uploaded files to ~/content/
df.head()

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Frozen Broccoli,0.48,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Frozen Corn,0.54,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Baked Potatoes,0.18,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
4,Tofu,0.93,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2


Our problem is minimisation (cost) with the following constraints:



1.   Fat must be between 20 and 50g
2.   Carbs between 130 and 200g
3. Fiber(sic.) between 60 and 125g
4. Protein between 100 and 150g
5. Total calories should be between 800 and 1300kcals

Specified mathematically the problem would be:
<br><br>
$Minimise \sum C_i \cdot F_i$
<br>$s.t.$<br>
$Cals_{lower} \leq Cals_i \cdot F_i \geq Cals_{higher}$ <br>
$Fat_{lower} \leq Fat_i \cdot F_i \geq Fat_{higher}$ <br>
$Carbs_{lower} \leq Carbs_i \cdot F_i \geq Carbs_{higher}$ <br>
$Protein_{lower} \leq Protein_i \cdot F_i \geq Protein_{higher}$ <br><br>

To get going we need to get the data setup to work with by making a list and a bunch of dictionaries:




In [4]:
# Create a list of the food items
food_items = list(df['Foods'])

# dictionary of costs per item
costs = dict(zip(food_items,df['Price/Serving']))

# further dictionaries for constraints
calories = dict(zip(food_items,df['Calories']))
fat = dict(zip(food_items,df['Total_Fat (g)']))
carbs = dict(zip(food_items,df['Carbohydrates (g)']))
fibre = dict(zip(food_items,df['Dietary_Fiber (g)'])) # spelling the UK/proper way
protein = dict(zip(food_items,df['Protein (g)']))

Next we start by specifying the model. We start with the overall function:

In [6]:
# Name the problem and the optimisation goal
problem = LpProblem("School_Dinners", LpMinimize)

Next we will add the function to be optimised and the constraints. To add the discrete list to the function (_food\_items_) we need to create a PuLP disctionary variable. Note we list it (for now) as continuous data)

In [7]:
# create a PuLP variable for the food items
food_vars = LpVariable.dicts("Food",food_items,0,cat='Continuous')

Now we can these to our problem, along with the constaints and objective function:

In [8]:
# always add the objective function first
problem += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost"

# constraints
problem += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0, "Min Cals"
problem += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0, "Max Cals"

problem += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "Min Fat"
problem += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "Max Fat"

# Carbs
problem += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "Min Carbs"
problem += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "Max Carbs"

# Fiber
problem += lpSum([fibre[f] * food_vars[f] for f in food_items]) >= 60.0, "Min Fiber"
problem += lpSum([fibre[f] * food_vars[f] for f in food_items]) <= 125.0, "Max Fibre"

# Protein
problem += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 100.0, "Min Protein"
problem += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 150.0, "Max Protein"

With this in place we can solve the problem! PuLP (handily) will pick the best solver for the problem type.

In [9]:
problem.solve()
print("Status:", LpStatus[problem.status])

Status: Optimal


"Optimal" status tells us PuLP has been able to sovle out problem (other outcomes could have been "infeasible" or "unbounded").

Finally we can print the final cost and mix of foods:

In [13]:
print(f"The total cost is: $ {round(value(problem.objective),2)} .")

print("The optimal (minimum cost) of the food consists of:\n")
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

The total cost is: $ 5.52 .
The optimal (minimum cost) of the food consists of:

Food_Frozen_Broccoli = 6.9242113
Food_Scrambled_Eggs = 6.060891
Food__Baked_Potatoes = 1.0806324


Problem solved. Let them eat broccoli, eggs and potatoes.

However, we may note that we end up with decimal places to work with. This may be harder for the staff to work with. Maybe it would work better if we restrict our values to integers.

Although this is is a considerably harder problem mathematically, it is fairly trivial to do in PuLP!

In [15]:
# convert to integer programming

# create a dictionary of integers rather than continuous data
food_integer = LpVariable.dicts("Food", food_items, 0, cat='Integer')

# respecify model
# Name the problem and the optimisation goal
problem_two = LpProblem("School_Dinners_Two", LpMinimize)

# always add the objective function first
problem_two += lpSum([costs[i]*food_integer[i] for i in food_items]), "Total Cost"

# constraints
problem_two += lpSum([calories[f] * food_integer[f] for f in food_items]) >= 800.0, "Min Cals"
problem_two += lpSum([calories[f] * food_integer[f] for f in food_items]) <= 1300.0, "Max Cals"

problem_two += lpSum([fat[f] * food_integer[f] for f in food_items]) >= 20.0, "Min Fat"
problem_two += lpSum([fat[f] * food_integer[f] for f in food_items]) <= 50.0, "Max Fat"

# Carbs
problem_two += lpSum([carbs[f] * food_integer[f] for f in food_items]) >= 130.0, "Min Carbs"
problem_two += lpSum([carbs[f] * food_integer[f] for f in food_items]) <= 200.0, "Max Carbs"

# Fiber
problem_two += lpSum([fibre[f] * food_integer[f] for f in food_items]) >= 60.0, "Min Fiber"
problem_two += lpSum([fibre[f] * food_integer[f] for f in food_items]) <= 125.0, "Max Fibre"

# Protein
problem_two += lpSum([protein[f] * food_integer[f] for f in food_items]) >= 100.0, "Min Protein"
problem_two += lpSum([protein[f] * food_integer[f] for f in food_items]) <= 150.0, "Max Protein"

# solve and print
problem_two.solve()
print("Status:", LpStatus[problem_two.status])

print(f"The total cost is: $ {round(value(problem_two.objective),2)} .")

print("The optimal (minimum cost) of the food consists of:\n")
for v in problem_two.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Status: Optimal
The total cost is: $ 5.58 .
The optimal (minimum cost) of the food consists of:

Food_Frozen_Broccoli = 7.0
Food_Raw_Lettuce_Iceberg = 1.0
Food_Scrambled_Eggs = 6.0
Food__Baked_Potatoes = 1.0


Now we have a slightly higher cost, but integer values for each (we even get a bit of lettuce thrown in :)

## Exercise
Update the code to add the following other constraints:


*   Cholesterol must be between 30 and 240mg
*   Sodium must be between 500 and 2000mg
*   Vitamin C must be between 400 and 5000mg

How does this change the result?
